In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
from transformers import BertModel, BertTokenizer
import joblib

# Load data comment 

In [6]:
df = pd.read_csv('data_crawler.csv', delimiter='\t', header=None)
print(df.shape)
# get all rows
# print(df[0]) 

(11, 1)


# Load Pretrain model BERT

In [7]:
'''
Load pretrain model/ tokenizers
'''
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#encode lines
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))
print('encode',tokenized[1])
# decode
print('decode',tokenizer.decode(tokenized[1]))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


encode [101, 2089, 14684, 2050, 5926, 7570, 4017, 25945, 7658, 2232, 1010, 18699, 2319, 2089, 20934, 10448, 6369, 16215, 2072, 20934, 10448, 9610, 13765, 18699, 2319, 9543, 18699, 2319, 5926, 7570, 4017, 25945, 7658, 2232, 1102, 9013, 10722, 1012, 25175, 18699, 2319, 2089, 16215, 2072, 15740, 24110, 12849, 3158, 1102, 2063, 21025, 1010, 1102, 2063, 16480, 1060, 6633, 10514, 29328, 2068, 27793, 2072, 27699, 2078, 1060, 6633, 7509, 1012, 1012, 1012, 1012, 2522, 1102, 17301, 1102, 5063, 2175, 2072, 12849, 8802, 4958, 2000, 2102, 1010, 9610, 1051, 14163, 2278, 17214, 15775, 2361, 18699, 2319, 29536, 2072, 12256, 6865, 13012, 27699, 25957, 2322, 16344, 1012, 12436, 2522, 1102, 17301, 22794, 2278, 6097, 2474, 13843, 7509, 2089, 25175, 21110, 12849, 2522, 7744, 6154, 1006, 8840, 4886, 8945, 2278, 9152, 7811, 25945, 24110, 2232, 6154, 1007, 11265, 2078, 17214, 16480, 1018, 1008, 2310, 11834, 11320, 5063, 4487, 2818, 24728, 1010, 2310, 2089, 16215, 2072, 14684, 2050, 1996, 18699, 2319, 1060, 33

# Fine tuning model and save model

In [8]:
#get all label 
labels = np.zeros(len(df[0]))
for i in range(len(df[0])):
    labels[i] = df[0][i][-1]
print('labels shape:', labels.shape)

# get lenght max of tokenized
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print('max len:', max_len)

# if lenght of tokenized not equal max_len , so padding value 0
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('padded:', padded[1])
print('len padded:', padded.shape)

#get attention mask ( 0: not has word, 1: has word)
attention_mask = np.where(padded ==0, 0,1)
print('attention mask:', attention_mask[1])

# Convert input to tensor
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)


# Train model
with torch.no_grad():
    last_hidden_states = model(padded, attention_mask =attention_mask)
#     print('last hidden states:', last_hidden_states)

features = last_hidden_states[0][:,0,:].numpy()
print('features:', features)

X_train, X_test, y_train, y_test = train_test_split(features, labels)

cl = LogisticRegression()
cl.fit(X_train, y_train)

# Save model
joblib.dump(cl, 'save_model.pkl')
sc = cl.score(X_test, y_test)
print('score:', sc)


ValueError: could not convert string to float: '?'